In [8]:
using Base.Test

In [33]:
module OffsetIndices


import Base: +, -, abs, *, /, div, convert, ==, <=, >=, show, to_index


immutable OffsetIndex{O, T <: Integer} <: Integer
    i::T
end

const ZeroIndex{T <: Integer} = OffsetIndex{-1, T}
const OneIndex{T <: Integer} = OffsetIndex{0, T}

raw(x::OffsetIndex) = x.i
raw(x::Integer) = x

function show{O, T}(io::IO, oi::OffsetIndex{O, T})
    print(io, "|$(raw(oi)) (indexes as $(T(OneIndex(oi))))|")
end

Base.eltype(::Type{OffsetIndex{O, T}}) where {O, T} = T
Base.eltype(oi::OffsetIndex) = eltype(typeof(oi))

# constructors and conversion
convert(::Type{OffsetIndex{O1, T1}}, x::OffsetIndex{O2, T2}) where {O1, O2, T1 <: Integer, T2 <: Integer} = 
    OffsetIndex{O1, T1}(raw(x) - O2 + O1)
convert(::Type{OffsetIndex{O}}, x::Integer) where {O} = OffsetIndex{O, eltype(x)}(x)
convert(::Type{OffsetIndex{O}}, x::OffsetIndex) where {O} = convert(OffsetIndex{O, eltype(x)}, x)
convert(::Type{OffsetIndex{O, T}}, x::Integer) where {O, T <: Integer} = OffsetIndex{O, T}(x)

function convert(::Type{IT}, x::OffsetIndex{O, T}) where {IT <: Integer, O, T <: Integer}
    if O == 0
        raw(x)
    else
        throw(InexactError())
    end
end

Base.@pure pure_max(x1, x2) = x1 > x2 ? x1 : x2
Base.promote_rule(::Type{T1}, ::Type{OffsetIndex{O, T2}}) where {T1 <: Integer, O, T2} = T1
Base.promote_rule(::Type{OffsetIndex{O1, T1}}, ::Type{OffsetIndex{O2, T2}}) where {O1, O2, T1, T2} = OffsetIndex{pure_max(O1, O2), promote_type(T1, T2)}

to_index(I::OffsetIndex) = OneIndex(I)
to_index(I::OffsetIndex{0}) = raw(I)

end

OI = OffsetIndices

OffsetIndices

In [34]:
# an array to index into
x = [1, 2, 3, 4];

In [44]:
# Creating an index from a regular integer:
o1 = OI.OneIndex(1)
@show o1
@test x[o1] == 1

o1 = |1 (indexes as 1)|


Test Passed


In [45]:
# Creating an index from zero-based data (like we would find
# in a .off mesh):
o0 = OI.ZeroIndex(0)
@show o0
@test x[o0] == 1

o0 = |0 (indexes as 1)|


Test Passed


In [37]:
# A one-based index can be converted to an integer
# because we have OneIndex(Int(x)) == x
@test Int(OI.OneIndex(1)) == 1

Test Passed


In [38]:
# But a non-one-based index would be imprecise:
@test_throws InexactError Int(OI.ZeroIndex(0))

Test Passed

      Thrown: InexactError

In [39]:
# We can read from some zero-based data and construct
# a one-based index without any explicit additions:
zero_based_data = [0, 1, 2]
desired_type = OI.OneIndex{Int}
@test convert.(desired_type, OI.ZeroIndex.(zero_based_data)) == OI.OneIndex.([1, 2, 3])

Test Passed


In [40]:
# Running a round trip from a zero-based input to a 
# zero-based output does no additions or subtractions:
@test OI.raw(OI.ZeroIndex(0)) == 0

Test Passed


In [46]:
# Converting from one offset to another preserves indexing:
o10 = convert(OI.OffsetIndex{10}, o1)
@show o10
@test x[o10] === x[o1]

o10 = |11 (indexes as 1)|


Test Passed


In [42]:
# ...but changes the raw representation:
@test OI.raw(o10) == 11

Test Passed
